In [3]:
from bs4 import BeautifulSoup
import urllib.request
import re
import mysql.connector
import os

In [ ]:
#connect to mysql server
cnx = mysql.connector.connect(host='152.19.68.141', user='ctolson', password='ilaYOU5!', database='sephora_cosmetics')
cursor = cnx.cursor()

#mysql insert functions
add_ingredient = ("INSERT INTO Product_Ingredient "
                    "(product_id, ingredient_name) "
                    "VALUES (%s, %s)")
add_review = ("INSERT INTO Reviews "
                    "(product_id, review, reviewer) "
                    "VALUES (%s, %s, %s)")
add_product = ("INSERT INTO Product "
                       "(name, brand, type, url) "
                       "VALUES (%s, %s, %s, %s)")


In [ ]:
search_topics = ['eyebrow-makeup-pencils','blush','eyeliner','eyeshadow','conditioner-hair','mascara','foundation-makeup', 'lipstick']     
search_topics = ['lipstick']

for search in search_topics:
    print(search)
    #scrape by search 
    link_main = 'http://www.sephora.com/'
    term = search+'?pageSize=-1'
    req = urllib.request.Request(link_main+term, headers={'User-Agent': 'Firefox'})
    webpage = urllib.request.urlopen(req).read()
    soup_web = BeautifulSoup(webpage, 'html.parser')
    
    #parse product urls
    product_url = soup_web.find_all('script', {'type': 'application/ld+json'})[1]
    product_url = str(product_url)
    product_url = re.findall("(url\":\")(.*?)(\"},{\")", product_url)
    product_url = [x[1] for x in product_url]


    #get product information
    for i in range(0, len(product_url)):
        try:   
            #scrap product page
            print(product_url[i])
            req_pg = urllib.request.Request(product_url[i], headers={'User-Agent': 'Chrome'})
            page = urllib.request.urlopen(req_pg).read()
            soup = BeautifulSoup(page, 'html.parser')
    
            #parse ingredients
            ingredients = soup.find_all('script', {'type': 'text/json'})[4]
            ingredients = str(ingredients)
            ingredients = re.sub("(,\"is_new\":true)", "", ingredients)
            ingredients = ingredients.split("ingredients\":")[1]
            ingredients = ingredients.split(",\"shipping_info")[0]

            #parse brand name
            brand = soup.find_all('script', {'type': 'text/json'})[4]
            brand = str(brand)
            brand = brand.split("brand_name\":")[1]
            brand = brand.split("},\"sku_number")[0]
            brand = re.split(',', brand)
            brand = str(brand)
            brand = brand[3:-3]

            #parse product name
            name = soup.find_all('script', {'type': 'text/json'})[4]
            name = str(name)
            name = name.split("display_name\":")[1]
            name = name.split(",\"variation_type")[0]
            name = re.split(',', name)
            name = str(name)
            name = name[3:-3]
            
            #insert product into mysql       
            data_product = (name, brand, search, str(product_url[i]))
            try:
                cursor.execute(add_product, data_product)
                product_id = cursor.lastrowid
            except:
                continue
                          
            #parse review page
            keywords = soup.find_all('meta')
            keywords = str(keywords)
            keywords = keywords.split('//reviews.sephora.com/static/')[1]
            keywords = keywords.split('/')[0]
            keywords2 = product_url[i].split('-')
            keywords2 = keywords2[len(keywords2)-1]
            link = 'http://reviews.sephora.com/'+keywords+'/'+keywords2+'/reviews.htm?format=embedded&amp;page=1m&amp;'
            req_pg2 = urllib.request.Request(link, headers={'User-Agent': 'Chrome'})
            page2 = urllib.request.urlopen(req_pg2).read()
            soup2 = BeautifulSoup(page2, 'html.parser')
            rg = soup2.find_all('span', {'class': 'BVRRNumber'})
            if rg == []:
                #no reviews
                reviews = 'NA'
                reviewer = 'NA'
            
                #insert reviews into mysql       
                data_review = (product_id, review, reviewer)
                try:
                    cursor.execute(add_review, data_review)
                except:
                    pass
            else:
                rg = str(rg).split("class=\"BVRRNumber\">")[1]
                rg = rg.split("</span")[0]
                rg = re.sub(",", "", rg)
                rg = int(int(rg)/5)
        
                #parse reviews
                for num in range(1,rg):
                    link = 'http://reviews.sephora.com/'+keywords+'/'+keywords2+'/reviews.htm?format=embedded&page='+str(num)+'&scrollToTop=true'
                    req_pg3 = urllib.request.Request(link, headers={'User-Agent': 'Chrome'})
                    page3 = urllib.request.urlopen(req_pg3).read()
                    soup3 = BeautifulSoup(page3, 'html.parser')
                    text = soup3.find_all('div', {'class': 'BVRRReviewTextContainer'})
                    user = soup3.find_all('span', {'class': 'BVRRNickname'})
                    for z in range(0,len(text)):
                        review = text[z].find_all('span', {'class': 'BVRRReviewText'})
                        review = str(review).split("class=\"BVRRReviewText\">")
                        review = re.sub("(</span>)", "", str(review))
                        review = re.sub("(<span)", "", str(review))
                        reviewer = str(user[z]).split("itemprop=\"author\">")[1]
                        reviewer = reviewer.split("</span")[0]
                        
                        #insert reviews into mysql       
                        data_review = (product_id, review, reviewer)
                        try:
                            cursor.execute(add_review, data_review)
                        except:
                            pass
                        
            #insert ingredients into mysql
            ingredients = str(ingredients).split('<br />', -1)[-1]
            ingredients = re.sub("\.\"", "", ingredients)
            ingredients = re.sub("<br>", "", ingredients)
            ingredients = re.sub("\n", "", ingredients)
            ingredients = re.sub("\r", "", ingredients)
            ingredients = ingredients.split(',')
            for ingredient_name in ingredients:
                data_ingredient = (product_id, ingredient_name)
                try:
                    cursor.execute(add_ingredient, data_ingredient)  
                except:
                    pass
            
            #commit entry    
            cnx.commit()
            print('commit')
        
        except:
            continue
        

lipstick
http://www.sephora.com/mind-game-velvet-lip-glide-set-P413929
http://www.sephora.com/give-me-some-nude-lip-P416228
http://www.sephora.com/everlasting-love-liquid-lipstick-P384954
http://www.sephora.com/rouge-interdit-satin-lipstick-P181912
http://www.sephora.com/huda-beauty-liquid-matte-lipstick-P411320
http://www.sephora.com/rouge-interdit-satin-lipstick-marble-rouge-revelateur-P416412
http://www.sephora.com/audacious-lipstick-P387906
http://www.sephora.com/cream-lip-stain-P281411
http://www.sephora.com/rouge-pur-couture-vernis-l-vres-vinyl-cream-P410743
http://www.sephora.com/velvet-matte-lip-pencil-duo-P408115
http://www.sephora.com/liquid-lipstick-P404831
http://www.sephora.com/amuse-bouche-lipstick-P405057
http://www.sephora.com/kiss-bliss-tarteist-creamy-matte-lip-paint-crayon-set-P411605
http://www.sephora.com/vice-lipstick-P409523
http://www.sephora.com/rouge-volupte-shine-oil-in-stick-lipstick-P377710
http://www.sephora.com/give-me-some-lip-liner-P413248
http://www.se

In [ ]:
#close mysql server
cnx.close()
